# How to deploy ADK Apps to Agent Engine

[Documentation link 🔗](https://google.github.io/adk-docs/deploy/agent-engine/)

## 1. Set the local environment variables
These will be used in the Agent Engine deployment

In [1]:
from dotenv import load_dotenv
import os

load_dotenv("../trends_and_insights_agent/.env")  # take environment variables

True

## 2. Set up the `AdkApp` Convenience wrapper for ADK Agents -> Agent Engine

Note the injection of the environment variables.

In [2]:
from vertexai.preview.reasoning_engines import AdkApp
from google.adk.artifacts import GcsArtifactService
from google.adk.sessions import VertexAiSessionService

from trends_and_insights_agent import agent

env_vars = {}

env_vars["GOOGLE_GENAI_USE_VERTEXAI"] = os.getenv("GOOGLE_GENAI_USE_VERTEXAI")
env_vars["BUCKET"] = os.getenv("BUCKET", "gs://default-bucket")
env_vars["GOOGLE_CLOUD_PROJECT_NUMBER"] = os.getenv("GOOGLE_CLOUD_PROJECT_NUMBER")
env_vars["YT_SECRET_MNGR_NAME"] = os.getenv("YT_SECRET_MNGR_NAME")
# env_vars["SESSION_STATE_JSON_PATH"] = os.getenv("SESSION_STATE_JSON_PATH")


def artifact_service_builder():
    return GcsArtifactService(bucket_name=env_vars["BUCKET"].split("gs://")[1])


my_agent = AdkApp(
    agent=agent.root_agent,
    enable_tracing=True,
    env_vars=env_vars,
    artifact_service_builder=artifact_service_builder,
)

INFO:root:

`SESSION_STATE_JSON_PATH`: None


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
/Users/jwortz/repos/zghost/zghost/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/Users/jwortz/repos/zghost/zghost/trends_and_insights_agent/common_agents/trend_assistant/tools.py:153: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return max_date.iloc[0][0].strftime("%m/%d/%Y")
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


# 3. Test the Agent Engine locally

In [11]:
session = my_agent.create_session(user_id="jwortz", session_id="12345")
session

Session(id='12345', app_name='default-app-name', user_id='jwortz', state={}, events=[], last_update_time=1752610640.748645)

In [12]:
my_agent.list_sessions(user_id="jwortz")

ListSessionsResponse(sessions=[Session(id='12345', app_name='default-app-name', user_id='jwortz', state={}, events=[], last_update_time=1752610640.748645)])

In [12]:
from pprint import pprint


def stream_agent(agent, prompt: str, session) -> None | Exception:
    try:
        session_id = session['id']
    # local prefers attributes vs. remote prefers dict keys
    except TypeError:
        session_id = session.id
    except Exception as e:
        return f"Session Object not valid: {e}"
        
    for event in agent.stream_query(
        user_id="jwortz",
        session_id=session_id,
        message=prompt,
    ):
        pprint(event)

In [14]:
first_prompt = f"Hello"
stream_agent(my_agent, first_prompt, session)

INFO:root:

Loading Initial State: {'state': {'artifact_keys': {'image_creatives': {}, 'video_creatives': {}}, 'target_product': 'PRS SE CE24 Electric Guitar', 'target_audience': ['Demographics: millennials who follow jam bands such as Widespread Panic and Phish.', 'Psychographics: millennials who respond positively to nostalgic messages.', 'Lifestyle or profession: frequent travelers; spending most income on concert experiences.', 'Hobbies, interests, humor: music lovers, attend lots of jam band concerts.', 'Actively researching concert and music festival tickets; musical instruments, love surreal memes (e.g.,  https://www.reddit.com/r/surrealmemes/).'], 'key_selling_points': ['Bolt-on Maple Neck - The bolt-on construction with a maple neck provides a distinct tone with enhanced brightness and a slightly more percussive attack', 'Wide Thin Profile- The Wide Thin neck carve offers a comfortable and fast playing experience, allowing for easy access to all 24 frets.', "Satin Finish - The

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'_state_init': True,
                             'artifact_keys': {'image_creatives': {},
                                               'video_creatives': {}},
                             'campaign_guide': {'brand': 'Paul Reed Smith '
                                                         '(PRS)',
                                                'campaign_name': 'Marketing '
                                                                 'Campaign '
                                                                 'Guide: PRS '
                                                                 'SE CE24 '
                                                                 'Guitar',
                                                'campaign_objectives': ['Increase '
                                                                        'Brand '
                                       

INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
Hello! I'm your AI Marketing Research & Strategy Assistant. I can help you with comprehensive insights, creative ideas, and trend analysis for your marketing campaigns. We'll start by populating your campaign guide and other relevant information, then dive into current trends, conduct in-depth research, and finally, generate some amazing ad creatives for you.

Let's begin by finding some interesting trends.

-----------------------------------------------------------
Function calls:
name: transfer_to_agent, args: {'agent_name': 'trends_and_insights_agent'}
-----------------------------------------------------------
Raw response:
{"sdk_http_response":{"headers":{"Content-Type":"application/json; charset=UTF-8","Vary":"Referer","Content-Encoding":"gzip","Date":"Tue, 15 Jul 2025 20:17:26 GMT","Server":"scaffolding on HTTPServer2","X-XSS-Protection":"0","X-Frame-Opt

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'request_count': 1,
                             'timer_start': 1752610644.2608}},
 'author': 'root_agent',
 'content': {'parts': [{'text': "Hello! I'm your AI Marketing Research & "
                                'Strategy Assistant. I can help you with '
                                'comprehensive insights, creative ideas, and '
                                'trend analysis for your marketing campaigns. '
                                "We'll start by populating your campaign guide "
                                'and other relevant information, then dive '
                                'into current trends, conduct in-depth '
                                'research, and finally, generate some amazing '
                                'ad creatives for you.\n'
                                '\n'
                                "Let's begin by finding some interesti

INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
None
-----------------------------------------------------------
Function calls:
name: get_daily_gtrends, args: {'today_date': '07/14/2025'}
-----------------------------------------------------------
Raw response:
{"sdk_http_response":{"headers":{"Content-Type":"application/json; charset=UTF-8","Vary":"Referer","Content-Encoding":"gzip","Date":"Tue, 15 Jul 2025 20:17:28 GMT","Server":"scaffolding on HTTPServer2","X-XSS-Protection":"0","X-Frame-Options":"SAMEORIGIN","X-Content-Type-Options":"nosniff","Alt-Svc":"h3=\":443\"; ma=2592000,h3-29=\":443\"; ma=2592000","Transfer-Encoding":"chunked"}},"candidates":[{"content":{"parts":[{"function_call":{"args":{"today_date":"07/14/2025"},"name":"get_daily_gtrends"}}],"role":"model"},"finish_reason":"STOP","avg_logprobs":-0.5055921454178659}],"create_time":"2025-07-15T20:17:27.554927Z","response_id":"V7d2aK_vIfy-gLUP6pjS

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'trends_and_insights_agent',
 'content': {'parts': [{'function_call': {'args': {'today_date': '07/14/2025'},
                                          'id': 'adk-654dc484-007c-4801-9635-e6833b5f721e',
                                          'name': 'get_daily_gtrends'}}],
             'role': 'model'},
 'id': 'hdvxlvjy',
 'invocation_id': 'e-b65dfeba-b3a4-45bf-9c8a-ba82f072f95b',
 'long_running_tool_ids': set(),
 'timestamp': 1752610646.852038,
 'usage_metadata': {'candidates_token_count': 19,
                    'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
                                                   'token_count': 19}],
                    'prompt_token_count': 1746,
                    'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
                                               'token_count': 1746}],
     

/Users/jwortz/zghost/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/Users/jwortz/zghost/trends_and_insights_agent/common_agents/trend_assistant/tools.py:134: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return max_date.iloc[0][0].strftime("%m/%d/%Y")
INFO:root:

max_date in trends_assistant: 07/14/2025


INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
INFO:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:

You are a helpful AI assistant, part of a multi-agent system designed for advanced web re

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'trends_and_insights_agent',
 'content': {'parts': [{'function_response': {'id': 'adk-654dc484-007c-4801-9635-e6833b5f721e',
                                              'name': 'get_daily_gtrends',
                                              'response': {'markdown_string_for_today_up_to_07/14/2025': '|    '
                                                                                                         '| '
                                                                                                         'term                            '
                                                                                                         '|   '
                                                                                                         'rank '
                                                                                          

I0000 00:00:1752610650.815693 3198072 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
Here are the top 25 trending Google Search terms:

| | term | rank | refresh_date |
|---:|:--------------------------------|-------:|:---------------|
| 1 | azealia banks | 1 | 2025-07-14 |
| 2 | oliver haarmann | 2 | 2025-07-14 |
| 3 | andrea gibson | 3 | 2025-07-14 |
| 4 | what time is the home run derby | 4 | 2025-07-14 |
| 5 | starbucks secret menu drinks | 5 | 2025-07-14 |
| 6 | flash flood warning | 6 | 2025-07-14 |
| 7 | joy taylor | 7 | 2025-07-14 |
| 8 | teddy bridgewater | 8 | 2025-07-14 |
| 9 | harry potter hbo series | 9 | 2025-07-14 |
| 10 | department of education | 10 | 2025-07-14 |
| 11 | garrett wilson | 11 | 2025-07-14 |
| 12 | jams peanut butter sandwich | 12 | 2025-07-14 |
| 13 | junior caminero | 13 | 2025-07-14 |
|

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'trends_and_insights_agent',
 'content': {'parts': [{'text': 'Here are the top 25 trending Google Search '
                                'terms:\n'
                                '\n'
                                '| | term | rank | refresh_date |\n'
                                '|---:|:--------------------------------|-------:|:---------------|\n'
                                '| 1 | azealia banks | 1 | 2025-07-14 |\n'
                                '| 2 | oliver haarmann | 2 | 2025-07-14 |\n'
                                '| 3 | andrea gibson | 3 | 2025-07-14 |\n'
                                '| 4 | what time is the home run derby | 4 | '
                                '2025-07-14 |\n'
                                '| 5 | starbucks secret menu drinks | 5 | '
                                '2025-07-14 |\n'
                                '| 6 | 

In [ ]:
followup_prompt = "Create images from these great ideas!"
stream_agent(my_agent, followup_prompt, session)

## 4. Deploy to Vertex AI Agent Engine
To call and manage agents in production, deploy the agent to Vertex AI Agent Engine.

**Important - run `poetry build` to package the agent**

In [3]:
! poetry build --format=wheel --output=notebooks/deployment

Building trends_and_insights_agent (0.1.0)
  - Building wheel
  - Built trends_and_insights_agent-0.1.0-py3-none-any.whl


#### Initialize the Vertex client, then create a `remote_agent` that is deployed to Vertex

This also takes the packaged agent code. This is required for more complex agents that have nested dependencies and require packaging

Also, before running - be sure to give Secret Manager access to the Agent Engine service account. This can simply be done by running `. setup_ae_sm_access.sh`.

The code can also be ran as follows (relative to repo root):

```bash
source trends_and_insights_agent/.env

export RE_SA="service-${GOOGLE_CLOUD_PROJECT_NUMBER}@gcp-sa-aiplatform-re.iam.gserviceaccount.com"
gcloud secrets add-iam-policy-binding "projects/$GOOGLE_CLOUD_PROJECT/secrets/$YT_SECRET_MNGR_NAME" \
  --member="serviceAccount:$RE_SA" \
  --role="roles/secretmanager.secretAccessor"
```

In [4]:
from vertexai import agent_engines # optional cleanup

for agent in agent_engines.list():
    agent.delete(force=True)

Delete Agent Engine backing LRO: projects/679926387543/locations/us-central1/operations/5470647007674630144


INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/679926387543/locations/us-central1/operations/5470647007674630144


Agent Engine deleted. Resource name: projects/679926387543/locations/us-central1/reasoningEngines/2101025983191580672


INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/679926387543/locations/us-central1/reasoningEngines/2101025983191580672


In [ ]:
import vertexai
from vertexai import agent_engines

GOOGLE_CLOUD_PROJECT = os.getenv("GOOGLE_CLOUD_PROJECT")
BUCKET = os.getenv("BUCKET")

vertexai.init(
    project=GOOGLE_CLOUD_PROJECT,
    location="us-central1",
    staging_bucket=BUCKET,
)

remote_agent = agent_engines.create(
    agent_engine=my_agent,
    display_name="trends-and-insights",
    description="You are a helpful AI assistant, part of a multi-agent system designed for advanced web research and ad creative generation.",
    requirements=[
        "deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl",
        "db-dtypes",
        "tabulate",
        "opencv-python"
    ],
    extra_packages=[
        "deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl",
    ],
    env_vars=env_vars,
)

Identified the following requirements: {'google-cloud-aiplatform': '1.104.0', 'pydantic': '2.11.7', 'cloudpickle': '3.1.1'}


INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.104.0', 'pydantic': '2.11.7', 'cloudpickle': '3.1.1'}


Failed to parse constraint: deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl. Exception: Expected end or semicolon (after name and no valid version specifier)
    deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl
              ^


    deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl
              ^


The following requirements are missing: {'google-cloud-aiplatform', 'pydantic', 'cloudpickle'}


...

Create AgentEngine backing LRO: projects/679926387543/locations/us-central1/reasoningEngines/2289614217587720192/operations/1794583811833462784


INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/679926387543/locations/us-central1/reasoningEngines/2289614217587720192/operations/1794583811833462784


View progress and logs at https://console.cloud.google.com/logs/query?project=wortz-project-352116


INFO:vertexai.agent_engines:View progress and logs at https://console.cloud.google.com/logs/query?project=wortz-project-352116


InvalidArgument: 400 Reasoning Engine resource [projects/679926387543/locations/us-central1/reasoningEngines/2289614217587720192] failed to start and cannot serve traffic. Please refer to our documentation (https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/troubleshooting/deploy) for checking logs and other troubleshooting tips. 3: Reasoning Engine resource [projects/679926387543/locations/us-central1/reasoningEngines/2289614217587720192] failed to start and cannot serve traffic. Please refer to our documentation (https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/troubleshooting/deploy) for checking logs and other troubleshooting tips.

### Save the agent engine resource name

This can be used by the command to deploy it to Agentspace

```bash
agent_engine = vertexai.agent_engines.get('projects/679926387543/locations/us-central1/reasoningEngines/1093257605637210112')
```

## 5. Try it remotely

In [10]:
online_session = remote_agent.create_session(user_id="jwortz")
online_session

{'appName': '2101025983191580672',
 'events': [],
 'userId': 'jwortz',
 'id': '2933798838848192512',
 'lastUpdateTime': 1753323314.45756,
 'state': {}}

#### Same idea above applies to remote agents

In [13]:
first_prompt = f"Hello"

stream_agent(remote_agent, first_prompt, online_session)

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'_state_init': True,
                             'brand': '',
                             'gcs_folder': '2025_07_24_02_15',
                             'img_artifact_keys': {'img_artifact_keys': []},
                             'key_selling_points': '',
                             'target_audience': '',
                             'target_product': '',
                             'target_search_trends': {'target_search_trends': []},
                             'target_yt_trends': {'target_yt_trends': []},
                             'vid_artifact_keys': {'vid_artifact_keys': []}}},
 'author': 'root_agent',
 'id': '9a016817-9bae-424d-a195-490e5578b19f',
 'invocation_id': 'e-f7a45520-39ec-4236-ada2-3c63a4c1168c',
 'timestamp': 1753323338.752489}
{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'request_count': 1,
   

In [37]:
second_prompt = f"hi" # select an arbitrary trend (#2 on the list for that day)
stream_agent(remote_agent, second_prompt, online_session)

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'trends_and_insights_agent',
 'content': {'parts': [{'text': 'Please choose a trending topic from the list '
                                'above.'}],
             'role': 'model'},
 'id': 'vXm1ERH4',
 'invocation_id': 'e-6bc3a0a6-b11c-427c-b515-484c5f737e28',
 'timestamp': 1752539837.354329,
 'usage_metadata': {'candidates_token_count': 10,
                    'candidates_tokens_details': [{'modality': 'TEXT',
                                                   'token_count': 10}],
                    'prompt_token_count': 1702,
                    'prompt_tokens_details': [{'modality': 'TEXT',
                                               'token_count': 1702}],
                    'thoughts_token_count': 74,
                    'total_token_count': 1786,
                    'traffic_type': 'ON_DEMAND'}}


# 6. Optional Cleanup

In [ ]:
remote_agent.delete(force=True)